In [1]:
import os
import re
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm.notebook import tqdm as log_progress
from pandas_profiling import ProfileReport
import plotly.express as px
from ast import literal_eval

In [2]:
import warnings
warnings.filterwarnings('ignore')

BASE_PATH = "E:\\data-science\\raw-data"
RES_PATH = "E:\\data-science\\datalake"

stocks_b3 = pd.read_csv(f"{BASE_PATH}\\b3\\traded-companies-info\\traded_companies_info.csv")
stocks_cvm = pd.read_csv(f"{BASE_PATH}\\cvm\\traded-companies-info\\cad_cia_aberta.csv")
bdr_b3 = pd.read_csv(f"{BASE_PATH}\\b3\\bdr-companies-info\\traded_companies_info.csv")
real_estate_b3 = pd.read_csv(f"{BASE_PATH}\\b3\\real-estate-info\\real-estate-info.csv")

## Leitura do dataset e análise dos atributos interessantes (Companhias abertas - B3)
___

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
stocks_b3.head(220)

,issuingCompany,companyName,tradingName,cnpj,industryClassification,industryClassificationEng,activity,website,hasQuotation,status,marketIndicator,market,institutionCommon,institutionPreferred,code,codeCVM,lastDate,otherCodes,hasEmissions,hasBDR,typeBDR,describleCategoryBVMF
0,RRRP,3R PETROLEUM ÓLEO E GÁS S.A,3R PETROLEUM,1.209181e+13,Petróleo. Gás e Biocombustíveis / Petróleo. Gá...,NaN,Exploração e produção de petróleo e gás natural.,NaN,NaN,A,18.0,NOVO MERCADO,ITAU CORRETORA ACOES,NaN,RRRP3,25291.0,06/09/2022 23:03:46,"[{'code': 'RRRP3', 'isin': 'BRRRRPACNOR5'}]",False,False,NaN,NaN
1,TZRD,3Z REALTY DESENVOLVIMENTO IMOBILIARIO S/A,3ZREALTY,8.145855e+12,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,EMPRESA,NaN,NaN,908830.0,06/09/2022 23:03:47,NaN,False,False,NaN,NaN
2,QVQP,524 PARTICIPACOES S.A.,524 PARTICIP,1.851771e+12,Outros / Outros / Outros,NaN,Holding - participação em outras sociedades,http://www.524participacoes.com.br/home/index....,NaN,A,7.0,SOMA,ITAU CORRETORA ACOES,NaN,QVQP3B,16284.0,06/09/2022 23:03:48,"[{'code': 'QVQP3B', 'isin': 'BRQVQPACNOR1'}]",False,False,NaN,NaN
3,GRDR,A GERADORA ALUGUEL DE MAQUINAS S.A.,GERADORA,3.384532e+13,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,EMPRESA,NaN,NaN,904110.0,06/09/2022 23:03:48,NaN,False,False,NaN,NaN
4,BODY,A BODYTECH PARTICIPACOES S.A.,BODYTECH,7.737623e+12,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,EMPRESA,NaN,NaN,901082.0,06/09/2022 23:03:49,NaN,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,HSBI,BANCO LOSANGO S/A - BM,LOSANGOBM,3.325432e+13,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,EMPRESA,NaN,NaN,904284.0,06/09/2022 23:08:06,NaN,False,False,NaN,NaN
216,LUSO,BANCO LUSO BRASILEIRO S/A,LUSOBRASILEI,5.911813e+13,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,EMPRESA,NaN,NaN,905091.0,06/09/2022 23:08:07,NaN,False,False,NaN,NaN
217,BMXM,BANCO MAXIMA S/A,MAXIMABC,3.392380e+13,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,EMPRESA,NaN,NaN,901017.0,06/09/2022 23:08:08,NaN,False,False,NaN,NaN
218,DCLS,BANCO MERCEDES BENZ DO BRASIL S/A,MERCEDESBBM,6.081419e+13,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,EMPRESA,NaN,NaN,902544.0,06/09/2022 23:08:09,NaN,False,False,NaN,NaN


In [5]:
stocks_b3.columns

Index(['issuingCompany', 'companyName', 'tradingName', 'cnpj',
       'industryClassification', 'industryClassificationEng', 'activity',
       'website', 'hasQuotation', 'status', 'marketIndicator', 'market',
       'institutionCommon', 'institutionPreferred', 'code', 'codeCVM',
       'lastDate', 'otherCodes', 'hasEmissions', 'hasBDR', 'typeBDR',
       'describleCategoryBVMF'],
      dtype='object')

In [6]:
stocks_b3.shape

(2197, 22)

In [7]:
stocks_b3[stocks_b3["hasBDR"] == True].head(4)

,issuingCompany,companyName,tradingName,cnpj,industryClassification,industryClassificationEng,activity,website,hasQuotation,status,marketIndicator,market,institutionCommon,institutionPreferred,code,codeCVM,lastDate,otherCodes,hasEmissions,hasBDR,typeBDR,describleCategoryBVMF
9,ABGD,ABRDN PHYSICAL GOLD SHARES ETF,ABDEN GOLD,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,ABGD39,50335.0,06/09/2022 23:03:58,"[{'code': 'ABGD39', 'isin': 'BRABGDBDR008'}]",False,True,DRE,Commodities
10,SIVR,ABRDN PHYSICAL SILVER SHARES ETF,ABDEN SILV,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,SIVR39,50327.0,06/09/2022 23:03:58,"[{'code': 'SIVR39', 'isin': 'BRSIVRBDR004'}]",False,True,DRE,Commodities
913,BFTS,FIRST TRUST CAPITAL STRENGTH ETF,FT CAPSTRENG,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,BFTS39,79740.0,06/09/2022 23:20:10,"[{'code': 'BFTS39', 'isin': 'BRBFTSBDR008'}]",False,True,DRE,Renda Variável
914,BKYY,FIRST TRUST CLOUD COMPUTING ETF,FT CLOUD CPT,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,BKYY39,79782.0,06/09/2022 23:20:10,"[{'code': 'BKYY39', 'isin': 'BRBKYYBDR006'}]",False,True,DRE,Renda Variável


In [8]:
stocks_b3["typeBDR"].value_counts()

DRE    156
DR3      5
DR1      2
DR2      1
Name: typeBDR, dtype: int64

In [9]:
stocks_b3["hasBDR"].value_counts()

False    1851
True      158
Name: hasBDR, dtype: int64

In [10]:
stocks_b3["describleCategoryBVMF"].unique()

array([nan, 'Commodities', 'Renda Variável', 'Renda Fixa'], dtype=object)

In [11]:
stocks_b3[stocks_b3["describleCategoryBVMF"] == "Commodities"]

,issuingCompany,companyName,tradingName,cnpj,industryClassification,industryClassificationEng,activity,website,hasQuotation,status,marketIndicator,market,institutionCommon,institutionPreferred,code,codeCVM,lastDate,otherCodes,hasEmissions,hasBDR,typeBDR,describleCategoryBVMF
9,ABGD,ABRDN PHYSICAL GOLD SHARES ETF,ABDEN GOLD,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,ABGD39,50335.0,06/09/2022 23:03:58,"[{'code': 'ABGD39', 'isin': 'BRABGDBDR008'}]",False,True,DRE,Commodities
10,SIVR,ABRDN PHYSICAL SILVER SHARES ETF,ABDEN SILV,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,SIVR39,50327.0,06/09/2022 23:03:58,"[{'code': 'SIVR39', 'isin': 'BRSIVRBDR004'}]",False,True,DRE,Commodities
1178,BIAU,ISHARES GOLD TRUST,GOLD TRUST,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,BIAU39,79006.0,06/09/2022 23:24:53,"[{'code': 'BIAU39', 'isin': 'BRBIAUBDR008'}]",False,True,DRE,Commodities
1236,BSLV,ISHARES SILVER TRUST,SILVER TRUST,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,1.0,BOLSA,NaN,BANCO B3 S.A.,BSLV39,79057.0,06/09/2022 23:25:40,"[{'code': 'BSLV39', 'isin': 'BRBSLVBDR002'}]",False,True,DRE,Commodities


## Filtragem e padronização dos campos

- Manutenção apenas das empresas com cadastro ativo (status = "A")
- Manutenção apenas dos registros de Ações, já que BDRs serão coletadas em outro contexto

In [12]:
names = {
    "Commodities": "Commodities", 
    "Renda Variável": "Ações",
    "BDR": "BDR"
}

cols = {
    "code": "id",
    "companyName": "name",
    "market": "trade_market",
    "describleCategoryBVMF": "market_type"
}

stocks = stocks_b3[(stocks_b3["status"] == "A") & (stocks_b3["code"].notna())]
stocks.loc[~stocks["typeBDR"].isna(), "describleCategoryBVMF"] = "BDR"
stocks["describleCategoryBVMF"] = stocks["describleCategoryBVMF"].fillna("Renda Variável").replace(names)
stocks = stocks[stocks["describleCategoryBVMF"].isin(names.values())]

stocks = stocks[cols.keys()].rename(columns=cols)

keep_stocks_mask = stocks["market_type"] == "Ações"
stocks = stocks[keep_stocks_mask]

In [13]:
stocks.head()

,id,name,trade_market,market_type
0,RRRP3,3R PETROLEUM ÓLEO E GÁS S.A,NOVO MERCADO,Ações
2,QVQP3B,524 PARTICIPACOES S.A.,SOMA,Ações
20,AERI3,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,NOVO MERCADO,Ações
23,AESB3,AES BRASIL ENERGIA S.A.,NOVO MERCADO,Ações
24,AESO3,AES BRASIL OPERAÇÕES S.A.,BOLSA,Ações


In [14]:
stocks_profile = ProfileReport(stocks, title="Stocks B3 profile report")

In [15]:
stocks_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

___
## STOCKS CVM

In [16]:
stocks_cvm.head()

,CNPJ_CIA,DENOM_SOCIAL,DENOM_COMERC,DT_REG,DT_CONST,DT_CANCEL,MOTIVO_CANCEL,SIT,DT_INI_SIT,CD_CVM,SETOR_ATIV,TP_MERC,CATEG_REG,DT_INI_CATEG,SIT_EMISSOR,DT_INI_SIT_EMISSOR,CONTROLE_ACIONARIO,TP_ENDER,LOGRADOURO,COMPL,BAIRRO,MUN,UF,PAIS,CEP,DDD_TEL,TEL,DDD_FAX,FAX,EMAIL,TP_RESP,RESP,DT_INI_RESP,LOGRADOURO_RESP,COMPL_RESP,BAIRRO_RESP,MUN_RESP,UF_RESP,PAIS_RESP,CEP_RESP,DDD_TEL_RESP,TEL_RESP,DDD_FAX_RESP,FAX_RESP,EMAIL_RESP,CNPJ_AUDITOR,AUDITOR
0,08.773.135/0001-00,2W ENERGIA S.A.,NaN,2020-10-29,2007-03-23,NaN,NaN,ATIVO,2020-10-29,25224,"Construção Civil, Mat. Constr. e Decoração",NaN,Categoria A,2020-10-29,FASE OPERACIONAL,2020-07-16,PRIVADO,SEDE,AV. ROQUE PETRONI JUNIOR,"Nº 1.089, SALA 1.101",JARDIM DAS ACáCIAS,SÃO PAULO,SP,NaN,4707000.0,11.0,39579400.0,11.0,39579499.0,ri@2wenergia.com.br,DIRETOR DE RELAÇÕES COM INVESTIDORES,EDUARDO URQUIZA FERNANDES PORTELADA,2022-05-10,AV. DOUTOR CHUCRI ZAIDAN 1.240,"SALA 2.701, 27º AN",VILA SÃO FRANCISCO,SÃO PAULO,SP,NaN,4707000.0,11,39579400.0,NaN,NaN,ri@2wenergia.com.br,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S
1,11.396.633/0001-87,3A COMPANHIA SECURITIZADORA,TRIPLO A COMPANHIA SECURITIZADORA,2010-03-08,2009-11-03,2015-12-18,Cancelamento Voluntário - IN CVM 480/09,CANCELADA,2015-12-18,21954,Securitização de Recebíveis,NaN,Categoria B,2010-03-08,FASE PRÉ-OPERACIONAL,2010-03-08,PRIVADO,SEDE,"Avenida Erasmo Braga, nº. 299",sala 703,Centro,RIO DE JANEIRO,RJ,BRASIL,20020000.0,21.0,22338867.0,21.0,22338867.0,dri@3asec.com.br,DIRETOR DE RELAÇÕES COM INVESTIDORES,FELIPE MARQUES DA FONSECA,2011-06-16,"AVENIDA ERASMO BRAGA, Nº. 299",SALA 703,CENTRO,RIO DE JANEIRO,RJ,NaN,20020000.0,21,22338867.0,21.0,22338867.0,juridico@triploasec.com.br,60.525.706/0001-07,MOORE STEPHENS LIMA LUCCHESI AUDITORES INDEPEN...
2,12.091.809/0001-55,3R PETROLEUM OLÉO E GÁS S.A.,NaN,2020-11-09,2010-06-08,NaN,NaN,ATIVO,2020-11-09,25291,Petróleo e Gás,BOLSA,Categoria A,2020-11-09,FASE OPERACIONAL,2020-08-31,PRIVADO,SEDE,"PRAIA DE BOTAFOGO, 186","SL 1301, 1401, 1501",BOTAFOGO,RIO DE JANEIRO,RJ,NaN,22250145.0,21.0,21455555.0,NaN,NaN,NaN,DIRETOR DE RELAÇÕES COM INVESTIDORES,RODRIGO PIZARRO LAVALLE DA SILVA,2020-08-13,RUA VISCONDE DE OURO PRETO,"5, SL 601",BOTAFOGO,RIO DE JANEIRO,RJ,NaN,22250180.0,21,21455555.0,0.0,0.0,ri@3rpetroleum.com.br,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.
3,01.547.749/0001-16,521 PARTICIPAÇOES S.A. - EM LIQUIDAÇÃO EXTRAJU...,521 PARTICIPAÇÕES S/A,1997-07-11,1996-07-30,NaN,NaN,ATIVO,1997-07-11,16330,Emp. Adm. Part. - Energia Elétrica,NaN,Categoria B,2011-04-28,LIQUIDAÇÃO EXTRAJUDICIAL,2015-04-30,PRIVADO,SEDE,"RUA DA ASSEMBLEIA, Nº 10, 37º ANDAR",3701,CENTRO,RIO DE JANEIRO,RJ,BRASIL,20011901.0,21.0,25560809.0,21.0,25560809.0,eximia@eximiacapital.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.596.945/0001-83,BAKER TILLY 4PARTNERS AUDITORES INDEPENDENTES S/S
4,01.851.771/0001-55,524 PARTICIPAÇOES SA,524 PARTICIPACOES SA,1997-05-30,1997-04-02,NaN,NaN,ATIVO,1997-05-30,16284,Emp. Adm. Part. - Sem Setor Principal,NaN,Categoria A,2010-01-01,FASE OPERACIONAL,1997-05-30,PRIVADO,SEDE,"AV. PRESIDENTE ANTôNIO CARLOS, Nº 51",10º ANDAR (PTE),CENTRO,RIO DE JANEIRO,RJ,NaN,20020010.0,21.0,38043700.0,21.0,38043480.0,gar@opportunity.com.br.,DIRETOR DE RELAÇÕES COM INVESTIDORES,NORBERTO AGUIAR TOMAZ,2022-05-06,AV. PRESIDENTE ANTÔNIO CARLOS,51,CENTRO,RIO DE JANEIRO,RJ,NaN,20020010.0,21,38043700.0,0.0,0.0,gar@opportunity.com.br,40.262.602/0001-31,"BKR - LOPES, MACHADO AUDITORES"


## Leitura do dataset e análise dos atributos interessantes (BDRs - B3)
___

In [17]:
bdr_b3.head(15)

,issuingCompany,companyName,tradingName,cnpj,industryClassification,industryClassificationEng,activity,website,hasQuotation,status,marketIndicator,market,institutionCommon,institutionPreferred,code,codeCVM,lastDate,otherCodes,hasEmissions,hasBDR,typeBDR,describleCategoryBVMF
0,MMMC,3M COMPANY,3M,NaN,Financeiro / Holdings Diversificadas / Holding...,NaN,NaN,www.3m.com,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,MMMC34,50504,23/09/2022 23:03:28,"[{'code': 'MMMC34', 'isin': 'BRMMMCBDR000'}]",False,True,DRN,NaN
1,A1BB,ABB LTD,ABB LTD,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,A1BB34,56464,23/09/2022 23:03:35,"[{'code': 'A1BB34', 'isin': 'BRA1BBBDR009'}]",False,True,DRN,NaN
2,ABTT,ABBOTT LABORATORIES,ABBOTT,NaN,Saúde / Medicamentos e Outros Produtos / Medic...,NaN,NaN,www.abbott.com,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,ABTT34,50512,23/09/2022 23:03:36,"[{'code': 'ABTT34', 'isin': 'BRABTTBDR007'}]",False,True,DRN,NaN
3,ABBV,ABBVIE INC.,ABBVIE,NaN,Consumo não Cíclico / Saúde / Medicamentos e O...,NaN,NaN,http://www.abbvie.com,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,ABBV34,51071,23/09/2022 23:03:36,"[{'code': 'ABBV34', 'isin': 'BRABBVBDR001'}]",False,True,DRN,NaN
4,A1BM,ABIOMED Inc,ABIOMED INC,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,A1BM34,55239,23/09/2022 23:03:38,"[{'code': 'A1BM34', 'isin': 'BRA1BMBDR006'}]",False,True,DRN,NaN
5,ABAM,ABN AMRO BANK N.V,ABN AMRO,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,14,BDR / CDA,NaN,BRADESCO,ABAM34,57762,23/09/2022 23:03:39,"[{'code': 'ABAM34', 'isin': 'BRABAMBDR002'}]",False,True,DRN,NaN
6,A2SO,ACADEMY SPORTS & OUTDOORS INC,ACADEMY SPOR,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,A2SO34,58980,23/09/2022 23:03:42,"[{'code': 'A2SO34', 'isin': 'BRA2SOBDR004'}]",False,True,DRN,NaN
7,ACNB,ACCENTURE PLC,ACCENTURE,NaN,Tecnologia da Informação / Programas e Serviço...,NaN,NaN,www.accenture.com,NaN,A,14,BDR / CDA,NaN,BRADESCO,ACNB34,50857,23/09/2022 23:03:42,"[{'code': 'ACNB34', 'isin': 'BRACNBBDR006'}]",False,True,DRN,NaN
8,ATVI,ACTIVISION BLIZZARD INC,ACTIVISION,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,ATVI34,51500,23/09/2022 23:03:46,"[{'code': 'ATVI34', 'isin': 'BRATVIBDR008'}]",False,True,DRN,NaN
9,ADBE,ADOBE INC,ADOBE INC,NaN,Não Classificados / Não Classificado / Não Cla...,NaN,NaN,NaN,NaN,A,14,BDR / CDA,NaN,BANCO B3 S.A.,ADBE34,51519,23/09/2022 23:03:49,"[{'code': 'ADBE34', 'isin': 'BRADBEBDR003'}]",False,True,DRN,NaN


In [18]:
bdr_b3.shape

(836, 22)

In [19]:
bdr_b3.dtypes

issuingCompany                object
companyName                   object
tradingName                   object
cnpj                         float64
industryClassification        object
industryClassificationEng    float64
activity                     float64
website                       object
hasQuotation                 float64
status                        object
marketIndicator                int64
market                        object
institutionCommon            float64
institutionPreferred          object
code                          object
codeCVM                        int64
lastDate                      object
otherCodes                    object
hasEmissions                    bool
hasBDR                          bool
typeBDR                       object
describleCategoryBVMF        float64
dtype: object

In [20]:
cols = {
    "code": "id",
    "companyName": "name",
    "market": "trade_market",
    "describleCategoryBVMF": "type"
}

bdr = bdr_b3[(bdr_b3["status"] == "A") & (bdr_b3["code"].notna())]
bdr = bdr[cols.keys()].rename(columns=cols)
bdr["market_type"] = "BDR"

In [21]:
bdr_profile = ProfileReport(bdr, title="Stocks B3 profile report")
stocks_profile.to_notebook_iframe()

## Leitura do dataset e análise dos atributos interessantes (Fundos imobiliários - B3)
___

In [22]:
real_estate_b3.head()

,segment,acronym,fundName,companyName,cnpj
0,NaN,CCME,FII CANUMA,CANUMA CAPITAL MULTIESTRATÉGIA FDO DE INV IMOB,NaN
1,NaN,ITIT,FII INTER IT,INTER TEVA INDICE DE TIJOLO FUNDO DE INVESTIM...,NaN
2,NaN,JASC,FII JASC REN,JASC RENDA VAREJO ESSENCIAL FDO DE INV IMOB -...,NaN
3,NaN,LAVF,FII LAVOURA,LAVOURA FUNDO DE INVESTIMENTO IMOBILIÁRIO,NaN
4,NaN,AFHI,FII AFHI CRI,AF INVEST CRI FDO. INV. IMOB - RECEBÍVEIS IMOB.,NaN


In [23]:
cols = {
    "acronym": "id",
    "companyName": "name"
}

real_estate = real_estate_b3[cols.keys()].rename(columns=cols)
real_estate["market_type"] = "FII"
real_estate["id"] = real_estate["id"].apply(lambda x: f"{x}11")
real_estate["trade_market"] = "BOLSA"          

In [24]:
real_estate.head()

,id,name,market_type,trade_market
0,CCME11,CANUMA CAPITAL MULTIESTRATÉGIA FDO DE INV IMOB,FII,BOLSA
1,ITIT11,INTER TEVA INDICE DE TIJOLO FUNDO DE INVESTIM...,FII,BOLSA
2,JASC11,JASC RENDA VAREJO ESSENCIAL FDO DE INV IMOB -...,FII,BOLSA
3,LAVF11,LAVOURA FUNDO DE INVESTIMENTO IMOBILIÁRIO,FII,BOLSA
4,AFHI11,AF INVEST CRI FDO. INV. IMOB - RECEBÍVEIS IMOB.,FII,BOLSA


In [25]:
real_estate_profile = ProfileReport(real_estate, title="Real Estate profile report")
real_estate_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Junção de ações, fundo imobiliário, bdr para composição do dataframe de renda variável
___

In [26]:
df = pd.concat([stocks, real_estate, bdr])
df.head()

,id,name,trade_market,market_type,type
0,RRRP3,3R PETROLEUM ÓLEO E GÁS S.A,NOVO MERCADO,Ações,NaN
2,QVQP3B,524 PARTICIPACOES S.A.,SOMA,Ações,NaN
20,AERI3,AERIS IND. E COM. DE EQUIP. GERACAO DE ENERGIA...,NOVO MERCADO,Ações,NaN
23,AESB3,AES BRASIL ENERGIA S.A.,NOVO MERCADO,Ações,NaN
24,AESO3,AES BRASIL OPERAÇÕES S.A.,BOLSA,Ações,NaN


## Definição de benchmark
___

In [27]:
benchmark = {
    "Ações": "IBOV",
    "BDR": "BDRX",
    "FII": "IFIX"
}

df["benchmark"] = df["market_type"].map(benchmark)

## Definição de estratégia
___

In [28]:
strategy = {
    "Ações": "Renda Variável",
    "BDR": "Internacional",
    "FII": "Alternativo"
}

df["strategy"] = df["market_type"].map(strategy)

## Definição de risco conforme a diretriz ANBIMA
___

In [29]:
risk = {
    "Ações": 4.0,
    "BDR": 4.0,
    "FII": 2.5
}

df["risk"] = df["market_type"].map(risk)

## Definição de atributos constantes para renda variável
___

In [30]:
df["liquidity"] = 2
df["has_periodic_income"] = True
df["is_income_tax_free"] = False
df["fees"] = ["brokerage"] * len(df)
df["market"] = "Renda Variável"
df["investor"] = "Público Geral"

In [31]:
products_profile = ProfileReport(df, title="Products profile report")

In [32]:
products_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Salvando dataframe de renda variável
___

In [33]:
df.to_parquet(f"{RES_PATH}/equities.snappy.parquet")